In [379]:
from sympy import *
from IPython.display import display, Math

In [380]:
#init_printing()

Resulting light = light coming from pixel * transmittance + integral over all points inbetween of (light coming from that point * transmittance from that point to camera)

Cryteks "exponential height fog" is used to simply scale the constants $\sigma_{a,c}$ and $\sigma_{s,c}$ depending on the height:

(Reformulated to divide by c instead. That way increasing c increases fade (more intuitive))

In [381]:
c, sigma_ac, sigma_sc = symbols('c sigma_ac sigma_sc')

In [382]:
def sigma_a(y):
    return sigma_ac*exp(-y/c)
def sigma_s(y):
    return sigma_sc*exp(-y/c)

In [383]:
def sigma_t(y):
    return sigma_a(y) + sigma_s(y)

Transmittance from pixel to camera:\
camera: $o$, pixel: $o+D*d$\
(in theory $o, d$ are vector quantities, but only the y component matters. so theyre treated as scalars in the following)

In [384]:
o, t, d, D = symbols('o t d D',infinite=False)

In [385]:
i = Integral(sigma_t(o+t*d), (t, 0, D))

In [386]:
exp(-i)

exp(-Integral(sigma_ac*exp((-d*t - o)/c) + sigma_sc*exp((-d*t - o)/c), (t, 0, D)))

Solving the integral

Simple in the case that d = 0

In [387]:
i0 = Integral(sigma_t(o), (t, 0, D))
i0

Integral(sigma_ac*exp(-o/c) + sigma_sc*exp(-o/c), (t, 0, D))

In [388]:
i0 = sigma_t(o)*Integral(1, (t, 0, D))
i0

(sigma_ac*exp(-o/c) + sigma_sc*exp(-o/c))*Integral(1, (t, 0, D))

In [389]:
simplify(i0)

D*(sigma_ac + sigma_sc)*exp(-o/c)

otherwise

In [390]:
d = symbols('d',nonzero=True, infinite=False)
i = Integral(sigma_t(o+t*d), (t, 0, D))
i

Integral(sigma_ac*exp((-d*t - o)/c) + sigma_sc*exp((-d*t - o)/c), (t, 0, D))

In [391]:
i.doit()

(-c*sigma_ac - c*sigma_sc)*exp((-D*d - o)/c)/d - (-c*sigma_ac - c*sigma_sc)*exp(-o/c)/d

In [392]:
display(Math('= \\frac{-(c*\\sigma_a + c*\\sigma_s)}{d} (e^{\\frac{-Dd-o}{c}} - e^{\\frac{-o}{c}})'))
display(Math('= \\frac{-(c*\\sigma_a + c*\\sigma_s)}{d} e^{\\frac{-o}{c}} (e^{\\frac{-Dd}{c}} - 1)'))
display(Math('= (c*\\sigma_a + c*\\sigma_s) e^{\\frac{-o}{c}} \\frac{(1 - e^{\\frac{-Dd}{c}})}{d}'))
display(Math('= (\\sigma_a + \\sigma_s) e^{\\frac{-o}{c}}c \\frac{(1 - e^{\\frac{-Dd}{c}})}{d}'))

<IPython.core.display.Math object>

<IPython.core.display.Math object>

<IPython.core.display.Math object>

<IPython.core.display.Math object>

In this form it is easy to see that for $d$->$0$ it converges to the result for the $d=0$ case\
(result if d=0):

In [393]:
simplify(i0)

D*(sigma_ac + sigma_sc)*exp(-o/c)

Evaluating the limit of the second part (can also be seen when graphed)

In [394]:
l = Limit(c*(1-exp(-D*d/c))/d, d, 0)
l

Limit(c*(1 - exp(-D*d/c))/d, d, 0)

In [395]:
l.doit()

D

The transformed version contains the same factors as the d=0 case, so in code those parts could be evaluated without branching.\
Sadly the transformed version is numerically unstable/inaccurate though and cant be used.

Next the 2nd part of the sum has to be calculated: Integrating the light reaching the camera from every point between the camera and the pixel

Again, first calculating the result for the case that $d=0$:

In [396]:
d = 0

In [397]:
def transmittanceIntegral(o,d,D):
    # return (-c*sigma_ac - c*sigma_sc)*exp((-D*d-o)/c)/d - (-c*sigma_ac - c*sigma_sc)*exp(-o/c)/d
    return D*(sigma_ac+sigma_sc)*exp(-o/c)

In [398]:
l_i, t_2 = symbols('l_i t_2')

In [399]:
#Helper function to calculate transmittance from the origin along a direction up until a given depth
def transmittance(o, d, D):
    return exp(-transmittanceIntegral(o,d,D))

In [400]:
def L_s(y): #in scatterd light at height
    return sigma_s(y)*l_i #just assume constant inscattering with a constanst phase function for this basic fog type

In [401]:
i0 = Integral(transmittance(o, d, t) * L_s(o+t*d), (t, 0, D))
i0

Integral(l_i*sigma_sc*exp(-o/c)*exp(-t*(sigma_ac + sigma_sc)*exp(-o/c)), (t, 0, D))

In [402]:
i0 = i0.doit()
i0

Piecewise((l_i*sigma_sc/(sigma_ac + sigma_sc) - l_i*sigma_sc*exp(-D*(sigma_ac + sigma_sc)*exp(-o/c))/(sigma_ac + sigma_sc), Ne(sigma_ac + sigma_sc, 0)), (D*l_i*sigma_sc*exp(-o/c), True))

The general case:

In [403]:
d = symbols('d',nonzero=True)

In [404]:
def transmittanceIntegral(o,d,D):
    return (-c*sigma_ac - c*sigma_sc)*exp((-D*d-o)/c)/d - (-c*sigma_ac - c*sigma_sc)*exp(-o/c)/d
    # return D*(sigma_ac+sigma_sc)*exp(-o/c)

In [405]:
i = Integral(transmittance(o, d, t) * L_s(o+t*d), (t, 0, D))
i

Integral(l_i*sigma_sc*exp((-d*t - o)/c)*exp(-(-c*sigma_ac - c*sigma_sc)*exp((-d*t - o)/c)/d + (-c*sigma_ac - c*sigma_sc)*exp(-o/c)/d), (t, 0, D))

In [406]:
i = i.doit()

In [407]:
i

Piecewise((-l_i*sigma_sc*exp(-(-c*sigma_ac - c*sigma_sc)*exp((-D*d - o)/c)/d + (-c*sigma_ac - c*sigma_sc)*exp(-o/c)/d)/(sigma_ac + sigma_sc) + l_i*sigma_sc/(sigma_ac + sigma_sc), Ne(sigma_ac + sigma_sc, 0)), (-c*l_i*sigma_sc*exp((-D*d - o)/c)/d + c*l_i*sigma_sc*exp(-o/c)/d, True))

Comparing both results:

In [410]:
i0.args[0][0]

l_i*sigma_sc/(sigma_ac + sigma_sc) - l_i*sigma_sc*exp(-D*(sigma_ac + sigma_sc)*exp(-o/c))/(sigma_ac + sigma_sc)

In [411]:
i.args[0][0]

-l_i*sigma_sc*exp(-(-c*sigma_ac - c*sigma_sc)*exp((-D*d - o)/c)/d + (-c*sigma_ac - c*sigma_sc)*exp(-o/c)/d)/(sigma_ac + sigma_sc) + l_i*sigma_sc/(sigma_ac + sigma_sc)

It becomes obvious that the result in both cases is simply:

In [414]:
T_r = symbols('T_r')
l_i*sigma_sc/(sigma_ac+sigma_sc) - l_i*sigma_sc*T_r/(sigma_ac+sigma_sc)

-T_r*l_i*sigma_sc/(sigma_ac + sigma_sc) + l_i*sigma_sc/(sigma_ac + sigma_sc)

Where $T_r$ is the total transmittance from the camera to the pixel, and $\frac{\sigma_{sc}}{\sigma_{ac}+\sigma_{sc}}$ is the usual volumetric albedo